<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/99_landsat_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

[Landsat 9](https://landsat.gsfc.nasa.gov/satellites/landsat-9) was successfully launched on Sept. 27, 2021. USGS has been providing Landsat data to the public since Feb. 10, 2022. Landsat 9 data can be downloaded from [EarthExplorer](https://earthexplorer.usgs.gov). The Earth Engine team has been ingesting Landsat 9 into the Public Data Catalog. As of Feb. 14, 2022, although Landsat 9 data have not been publicly listed on the [Earth Engine Datasets](https://developers.google.com/earth-engine/datasets) page, you can access the data through `ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')` .

In [ ]:
# !pip install geemap

Import libraries.

In [ ]:
import ee
import geemap

Create an interactive map.

In [ ]:
Map = geemap.Map()

Find out how many Landsat imgaes are available.

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
print(collection.size().getInfo())

Create a median composite.

In [ ]:
median = collection.median()

Apply scaling factors. See https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

In [ ]:
def apply_scale_factors(image):
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermalBands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [ ]:
dataset = apply_scale_factors(median)

Specify visualization parameters.

In [ ]:
vis_natural = {
    "bands": ["SR_B4", "SR_B3", "SR_B2"],
    "min": 0.0,
    "max": 0.3,
}

vis_nir = {
    "bands": ["SR_B5", "SR_B4", "SR_B3"],
    "min": 0.0,
    "max": 0.3,
}

Add data layers to the map.

In [ ]:
Map.addLayer(dataset, vis_natural, "True color (432)")
Map.addLayer(dataset, vis_nir, "Color infrared (543)")
Map

![](https://i.imgur.com/USPMXzw.png)

Create linked maps for visualizing images with different band combinations. For more information on common band combinations of Landsat 8/9, see https://gisgeography.com/landsat-8-bands-combinations/

Specify visualization parameters.

In [ ]:
vis_params = [
    {"bands": ["SR_B4", "SR_B3", "SR_B2"], "min": 0, "max": 0.3},
    {"bands": ["SR_B5", "SR_B4", "SR_B3"], "min": 0, "max": 0.3},
    {"bands": ["SR_B7", "SR_B6", "SR_B4"], "min": 0, "max": 0.3},
    {"bands": ["SR_B6", "SR_B5", "SR_B2"], "min": 0, "max": 0.3},
]

Specify labels for each layers.

In [ ]:
labels = [
    "Natural Color (4, 3, 2)",
    "Color Infrared (5, 4, 3)",
    "Short-Wave Infrared (7, 6 4)",
    "Agriculture (6, 5, 2)",
]

Create linked maps.

In [ ]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[40, -100],
    zoom=4,
    ee_objects=[dataset],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

![](https://i.imgur.com/c4FsGBI.png)

Create a split-panel map for comparing Landsat 8 and 9 images.

Retrieve two sample images.

In [ ]:
landsat8 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_015043_20130402")
landsat9 = ee.Image("LANDSAT/LC09/C02/T1_L2/LC09_015043_20211231")

Apply scaling factors.

In [ ]:
landsat8 = apply_scale_factors(landsat8)
landsat9 = apply_scale_factors(landsat9)

Generate Earth Engine layers.

In [ ]:
left_layer = geemap.ee_tile_layer(landsat8, vis_natural, "Landsat 8")
right_layer = geemap.ee_tile_layer(landsat9, vis_natural, "Landsat 9")

Create a split-panel map.

In [ ]:
Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map

![](https://i.imgur.com/i6lUYHF.png)